In [1]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(open('Prod223_2624_00390223_20191031.html'),'html.parser')

# for tag in soup.find_all(re.compile('ix:*')):
#     print(f"tag name {tag.name}")
#     for k,v, in tag.attrs.items():
#         print(f"k {k}")
#         print(f"v {v}")
#         print(f"type v {type(v)}")
        
#     print(tag.text)

In [2]:
#e.g file name = 'Prod223_2624_00390223_20191031.html'

In [3]:
def pageParser(file_name):
    
    from bs4 import BeautifulSoup
    import re
    soup = BeautifulSoup(open(file_name),'html.parser')
    
    web_page_as_dictionary = {}
    
    ix_tags = ['ix:nonfraction','ix:references','ix:resources','ix:nonnumeric','ix:header','ix:hidden']
    
    discovered_ix_tags = list(set(tag.name for tag in soup.find_all(re.compile('ix:*'))))
    
    for item in discovered_ix_tags:
        if item not in ix_tags:
            print("warning: ix tags not accounted for in this script have been identified...")
            
    
    web_page_as_dictionary['ix:nonnumeric'] =  findRelevantIXResults(soup.find_all('ix:nonnumeric'))
    web_page_as_dictionary['ix:nonfraction'] =  findRelevantIXResults(soup.find_all('ix:nonfraction'))
    web_page_as_dictionary['ix:resources'] =   findRelevantXBRLIResults(soup.find_all(re.compile('xbrli*')))
    
    
    #results in ['ix:hidden'] are included in the ['ix:nonnumeric'], ['ix:nonfraction'] results
    #results in ['ix:header'] are included in the ['ix: resources'] results
    
   
#     for ix_tag in ix_tags:
#         if ix_tag == "ix:nonnumeric" or ix_tag == "ix:nonfraction":
            
#             try:
#                 web_page_as_dictionary[ix_tag] == findRelevantIXResults(soup.find_all(ix_tag))
#                 break
#             except Exception as e:
#                 print(f"error: {e}")

    return web_page_as_dictionary
    
    
    
    

In [4]:
def findRelevantIXResults(items):
    results = {}
    # second_level = {}
    for item in items:
       


        second_level = {}
        item_copy = item
     
        subitem_list = []
       
        for subitem in item:
         
            subitem_list.append(subitem)

        second_level.update(item.attrs)

        second_level.update({"description":subitem_list})
        
        table_in_description = False
        
        description = ""
        
        for element in second_level['description']:

            try:
            
                description += element.text
          
                table_in_description = True


            except:
                
                try:
              
                    description += element.find('tr').text
                    table_in_description = True

                except:
                      try: 
                       
                        description += element.find('span',class_='ta_10').text
                        table_in_description == True
                      except:
                        True

        if table_in_description == True:
                  second_level['description'] = description


        try:
            if second_level['format'] == 'ixt2:nocontent':
                second_level['description'] = 'Missing'

        except:
            True
        if 'name' in item.attrs.keys():
            del second_level['name']
            
            if item.attrs['name'] not in results.keys():
                results[item.attrs['name']] = []
            else:
                results[item.attrs['name']].append("and")
                
            results[item.attrs['name']].append(second_level)

  
      
      
    return results

In [5]:
def findRelevantXBRLIResults(items):
    XBRLI_dictionary = {}
#     items =soup.find_all(re.compile('xbrli*'))
    for tag in items:
        second_level = {}
        for k, v in tag.attrs.items():
            new_tag_text = tag.text

            try:

                matchy = re.search(r'ns[0-9]*:.+?[0-9]{4}-',tag.text)

                matchy = matchy.group(0)[:len(matchy.group(0))-5]

                remainder = tag.text.replace(matchy," ")

                new_tag_text = (remainder.split() + [matchy])

            except Exception as e:
                True
#                 print(e)

            second_level['description'] =  new_tag_text



            second_level['type'] = k


#                
            if v not in XBRLI_dictionary.keys():
                XBRLI_dictionary[v] = []
            else:
                XBRLI_dictionary[v].append("and")
            XBRLI_dictionary[v].append(second_level)
#    

    return XBRLI_dictionary

In [6]:
test =pageParser('Prod223_2624_00390223_20191031.html')

In [9]:
test.keys()

dict_keys(['ix:nonnumeric', 'ix:nonfraction', 'ix:resources'])

In [12]:
import pprint
for key in test.keys():
    pprint.pprint(test[key]) 
    

{'ns10:AccountingStandardsApplied': [{'contextref': 'AccountingStandardsAppliedHypercube_FY_31_10_2019_Set5',
                                      'description': 'Missing',
                                      'format': 'ixt2:nocontent',
                                      'xmlns:ix': 'http://www.xbrl.org/2013/inlineXBRL'}],
 'ns10:AccountsStatusAuditedOrUnaudited': [{'contextref': 'AccountsStatusHypercube_FY_31_10_2019_Set6',
                                            'description': 'Missing',
                                            'format': 'ixt2:nocontent',
                                            'xmlns:ix': 'http://www.xbrl.org/2013/inlineXBRL'}],
 'ns10:AccountsTypeFullOrAbbreviated': [{'contextref': 'AccountsTypeHypercube_FY_31_10_2019_Set9',
                                         'description': 'Missing',
                                         'format': 'ixt2:nocontent',
                                         'xmlns:ix': 'http://www.xbrl.org/2013/inlineXBRL'}